In [17]:

import torch
import torch.nn as nn
import torch.nn.functional as F

import openpyxl

# from torch_geometric import utils
import networkx as nx
import pandas as pd
# from torch_geometric.nn import GCNConv
# from torch_geometric.data import Data
import pygraphviz as pgv
import numpy as np
from node2vec import Node2Vec
import pickle
import matplotlib.pyplot as plt
from PIL import Image
# Image.MAX_IMAGE_PIXELS = 3075432330


# In[5]:

In [18]:
graph_source = pd.read_csv('to_graph.csv')
df = pd.read_csv('final_result_mibig.csv')
df_co_net = pd.read_excel('Microbe_microbe_co_network.xls')

In [19]:
i = df_co_net['species1']
i2 = df_co_net['species2']

In [20]:
i_new = []
i2_new = []
for idx in i:
     i_new.append(idx.split('_'))
for idx in i2:
    i2_new.append(idx.split('_'))


In [21]:
# check if any overlaps between MCCN and co-occurence network
MCCN_org = graph_source['organism_name']

MCCN_org_new = []
for idx in MCCN_org:
    MCCN_org_new.append(idx.split(' '))

In [22]:
similar  = []
similar_2 = []
for idx in i_new:
    for idx_1 in MCCN_org_new:
        if idx[0] in idx_1:
            similar.append((idx, idx_1))
for idx in i2_new:
    for idx_1 in MCCN_org_new:
        if idx[0] in idx_1:
            similar_2.append((idx, idx_1))

In [23]:
similar_ = np.array(similar)
similar2_ = np.array(similar_2)

similar_all = np.append(similar_, similar2_, axis=0)

similar_all_list = similar_all.tolist()


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [24]:
def get_unique_list(seq):
    seen = []
    return [x for x in seq if x not in seen and not seen.append(x)]
dereplicate_all_new = get_unique_list(similar_all_list)
# dereplicate_2 = get_unique_list(similar_2)
print(len(dereplicate_all_new))
# print(dereplicate_2)
# export similar similar2 to csv

621


In [61]:
import csv
f = open('final_all.csv', 'w', newline='')
writer = csv.writer(f)
writer.writerows(dereplicate_all_new)
f.close()


In [25]:



# In[13]:


graph_source


# In[24]:


node_organism = list(graph_source['organism_name'])
node_organism_colored = []

node_compound = list(graph_source['compound'])
node_compound_colored = []

edges = []
for k in range(len(node_organism)):
    edges.append((node_organism[k], node_compound[k]))


# In[27]:
# organism ---> compound
pos = {}
#pos.update((node, (0, index)) for index, node in enumerate(node_organism))
#pos.update((node, (1, index)) for index, node in enumerate(node_compound))
G = nx.DiGraph()
G.add_nodes_from(node_organism, bipartite=0, color='red')
G.add_nodes_from(node_compound, bipartite=1, color='green')
G.add_edges_from(edges)
print(len(node_compound))
print(len(node_organism))
degree_organism_G = np.array(G.degree(get_unique_list(node_organism)))
degree_compound_G = np.array(G.degree(get_unique_list(node_compound)))
print('organism nodes:', len(get_unique_list(node_organism)))
print('compound nodes:', len(get_unique_list(node_compound)))
print('mean degrees of old MCCN compounds: ', np.mean(degree_compound_G[:,-1].astype(float)))
print('mean degrees of old MCCN organism:', np.mean(degree_organism_G[:,-1].astype(float)))

2689
2689
organism nodes: 1176
compound nodes: 2345
mean degrees of old MCCN compounds:  1.100639658848614
mean degrees of old MCCN organism: 2.1947278911564627


In [26]:
modified_all_new = []
for i in dereplicate_all_new:
    modified_all_new.append([' '.join(i[0]),' '.join(i[1])])

In [27]:
new_org = []
for i in modified_all_new:
    new_org.append(i[0])


In [28]:
G_new_micbio = G
for i in list(G_new_micbio.nodes):
    for j in modified_all_new:
        if j[-1] == i:
            G_new_micbio.add_edge(j[0], i)
            # print(j[0], i, j[-1])


In [29]:
degree_organism = G_new_micbio.degree(get_unique_list(node_organism))
degree_compound = G_new_micbio.degree(get_unique_list(node_compound))
degree_new_org = G_new_micbio.degree(get_unique_list(new_org))
degree_compound = np.array(degree_compound)
degree_organism = np.array(degree_organism)
degree_new_org = np.array(degree_new_org)

In [30]:
G
G_new_micbio

In [31]:
print('new_org nodes:', len(get_unique_list(new_org)))
print('mean degrees of compounds: ', np.mean(degree_compound[:,-1].astype(float)))
print('mean degrees of organism:', np.mean(degree_organism[:,-1].astype(float)))
print('mean degrees of new_organism:', np.mean(degree_new_org[:,-1].astype(float)))

new_org nodes: 42
mean degrees of compounds:  1.100639658848614
mean degrees of organism: 2.7253401360544216
mean degrees of new_organism: 15.238095238095237


In [ ]:
G
G_new_micbio
nx.write_graphml_lxml(G_new_micbio, "MCCN_new_org.graphml")





In [101]:
# node2vec on graph
node2vec = Node2Vec(G_new_micbio)
# Embed
model = node2vec.fit()  # Any keywords acceptable by gensim.Word2Vec can be passed, `diemnsions` and `workers` are automatically passed (from the Node2Vec constructor)

# Save the model of Node2vec
with open('o2c_new_bio.pickle', mode='wb') as f:
    pickle.dump(model, f)
print('END')

Computing transition probabilities:   0%|          | 0/3562 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 10/10 [00:00<00:00, 12.13it/s]


END


In [6]:
# applied Graph convolution network to learn features
if torch.cuda.is_available():
    device = 'cuda'
print('use'+device)
def representation_learning(input_dim, hidden_dim, output_dim, data):
    dropout = 0.5
    convs = nn.ModuleList()
    convs.append(GCNConv(input_dim, hidden_dim))
    num_layers = 3
    heads = 1
    for l in range(num_layers-1):
        convs.append(GCNConv(heads * hidden_dim, hidden_dim))

    post_mp = nn.Sequential(
        nn.Linear(heads * hidden_dim, hidden_dim), nn.Dropout(dropout),
        nn.Linear(hidden_dim, output_dim))


    x, edge_index, batch = data.x, data.edge_index, data.batch

    for i in range(num_layers):
        x = convs[i](x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=dropout)

    x = post_mp(x)
    return x, F.log_softmax(x, dim=1)


usecuda


In [7]:
G_1 = nx.convert_node_labels_to_integers(G)

In [8]:
node_organism_label = np.ones([1, len(node_organism) ])
node_compound_label = np.zeros([1, len(node_compound) ])
print('compound length: ', len(node_compound))
print('organism length', len(node_organism))


Data1 = utils.from_networkx(G_1)
# node_compound.append(node_organism)
x = torch.tensor([node_compound_label, node_organism_label], dtype=torch.float)
print(x.shape)
print(x)
x =x.view(-1, 5378).t()
print(x.shape)
print(x)
edges_list = list(G_1.edges)
edge_index = torch.tensor(edges_list, dtype=torch.long)
data = Data(x=x, edge_index=edge_index.t().contiguous())

hidden_dim = 128
data.edge_index
print(data)
print(data.num_node_features)
a, b = representation_learning(data.num_node_features, hidden_dim, 128, data)

compound length:  2689
organism length 2689
torch.Size([2, 1, 2689])
tensor([[[0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.]]])
torch.Size([5378, 1])
tensor([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])
Data(x=[5378, 1], edge_index=[2, 2581])
1


C:\Users\asse9\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:210.)
  if __name__ == '__main__':


In [9]:
data.x

tensor([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])

In [10]:
a = a.to('cpu').detach().numpy().copy()
b = b.to('cpu').detach().numpy().copy()

In [11]:
print(a.shape)
print(b.shape)
print(a[0].max(), a[0].min())
print(b[0].max(), b[0].min())
a_compound = a[0:2689]
print(a_compound.shape)
np.save('o2c_gcn.npy', a_compound)

(5378, 128)
(5378, 128)
0.15697138 -0.19782788
-4.699101 -5.0539002
(2689, 128)


In [12]:
from node2vec import Node2Vec

node2vec = Node2Vec(G)
# Embed
model = node2vec.fit()

Computing transition probabilities:   0%|          | 0/3521 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 10/10 [00:00<00:00, 13.94it/s]


In [13]:
# convert node2vec to dataframe
emb_df = (
    pd.DataFrame(
        [model.wv.get_vector(str(n)) for n in G.nodes()],
        index = G.nodes
    )
)
emb_df.shape
print(emb_df[0].max, emb_df[0].min)

<bound method NDFrame._add_numeric_operations.<locals>.max of Verrucosispora maris AB-18-032    0.004781
Kutzneria albida DSM 43870        0.000723
Alternaria alternata              0.002823
Aspergillus oryzae                0.001534
Aspergillus parasiticus           0.001505
                                    ...   
perquinoline A                    0.004247
perquinoline B                    0.005060
perquinoline C                   -0.001977
ilicicolin H                     -0.001801
dehydrofosmidomycin              -0.000834
Name: 0, Length: 3521, dtype: float64> <bound method NDFrame._add_numeric_operations.<locals>.min of Verrucosispora maris AB-18-032    0.004781
Kutzneria albida DSM 43870        0.000723
Alternaria alternata              0.002823
Aspergillus oryzae                0.001534
Aspergillus parasiticus           0.001505
                                    ...   
perquinoline A                    0.004247
perquinoline B                    0.005060
perquinoline C      

In [14]:
## check if the microbe-compound interaction graph is directed graph and the directions are right
## also check number of edges
T_edge = 0
F_edge = 0
a = []
for k, v in enumerate(node_organism):
    a.append((v, node_compound[k]))
    if G.has_edge(v, node_compound[k]) == True:
        T_edge +=1
    if G.has_edge(node_compound[k], v) == True:
        F_edge +=1
print(G.number_of_edges())
print(T_edge)
print(F_edge)

2581
2689
0


In [15]:
seen = set()
dupes = []

for x in a:
    if x in seen:
        dupes.append(x)
    else:
        seen.add(x)

In [16]:
len(dupes)

108